In [16]:
import torch
import torch.utils.data as data

<center>

# DataLoaders
</center>


## utils.data.DataLoader()

```python
torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    sampler=None,
    batch_sampler=None,
    num_workers=0,
    collate_fn=None,
    pin_memory=False,
    drop_last=False,
    timeout=0,
    worker_init_fn=None,
    multiprocessing_context=None,
)
```

**Docstrings**

`DataLoader`集成了数据集和采样器，同时确保了给定数据集的可迭代性，`DataLoader`支持 map-style 和 iterable-style 的数据集，并支持单进程或多进程加载、customizing loading order and optional automatic batching(collation) 和 memory pinning

详见`torch.utils.data`模块的说明文档

个人解读：如其名称，数据加载器，能够在神经网络模型中各种数据加载的操作

**Args**

- dataset: 所提供的数据集

- batch_size: 每一batch加载的数据量

- shuffle: 每一 epoch 是否 shuffle

- sampler: 定义从数据集采样的策略，若此参数未指明，则`shuffle`必须为``False``.

- batch_sampler: 类似`sampler`，单一次返回一batch的索引，与`batch_size`, `shuffle`, `sampler`, `drop_last`互斥

- num_workers: 用于定义数据由多少子进程加载；0表示数据集由主进程加载

- collate_fn (callable): 将样本合并形成一个小批tensor，常在使用批处理加载 map-style 的数据集时使用

- pin_memory: ``True``时 data loader会将 tensor 复制到 CUDA 的固定内存计算，随后再返回计算值；如果数据元素是自定义类型，或者`collate_fn`返回的批处理是自定义类型，请参见下面的示例（然而说明文档后面没有写示例）

- drop_last: ``True``时将最后不完整的 batch 丢掉

- timeout: if positive, the timeout value for collecting a batch from workers. Should always be non-negative. (default: ``0``)

- worker_init_fn (callable): 若不为``None``，则会在每一个子进程的 seeding 之后、数据加载之前，通过进程的 id 调用该函数

- warning: If the ``spawn`` start method is used, `worker_init_fn` cannot be an unpicklable object, e.g., a lambda function. See `multiprocessing-best-practices` on more details related to multiprocessing in PyTorch.

- note: ``len(dataloader)`` heuristic is based on the length of the sampler used. When `dataset` is an `~torch.utils.data.IterableDataset`, ``len(dataset)`` (if implemented) is returned instead, regardless of multi-process loading configurations, because PyTorch trust user `dataset` code in correctly handling multi-process loading to avoid duplicate data. See `Dataset Types`_ for more details on these two types of datasets and how `~torch.utils.data.IterableDataset` interacts with `Multi-process data loading`_.

# 

# 

<center>

# Datasets
</center>

## data.Dataset()
`data.Dataset(*args, **kwds)`

**Docstring**

表示一个数据集的抽象类，所有可以表示为键到样本的映射的数据集都应继承它，这里键也可包括索引值；继承`Dataset`的子类都应重写`__getitem__`方法，以根据给定的键来获取数据；并根据需要重写`__len__`方法，由于该方法常由`data.Sampler`、`data.DataLoader`等实例对象调用以获得数据集的大小，进而建议所有继承此类的数据集均对`__len__`方法进行实现；需要注意的是，`data.DataLoader`默认构造一个生成整数型索引的索引采样器，若需要使其能利用非正整数索引或键值进行映射，需提供自定义采样器

**File**:         \torch\utils\data\dataset.py

**Type**:           type

## data.IterableDataset()
`data.IterableDataset(*args, **kwds)`

可迭代式的数据集的基类；所有可以表示为数据的可迭代对象的数据集都应该继承它，尤其对于数据以流形式加载的情况；继承的子类应对`__iter__`方法进行重写，该方法应返回该数据集样本组成的可迭代对象；

当其子类与`DataLoader`一起使用时，数据集中的每个项都会从`DataLoader`的迭代器产生；当`num_workers > 0`时，每个工作进程均含有一个互不相同的数据集的副本，进而通常需要对每个副本进行独立配置，以避免从 worker 中返回重复的数据；在工作进程中调用`dataget_worker_info`可返回有关该工作进程的信息，该进程信息可在数据集的`__iter__`方法中使用或作为`DataLoader`的`worker_init_fn`参数，以调整每个副本的行为


**Type**:           type

**Subclasses**:     ChainDataset

### Example

In [ ]:
# splitting workload across all workers in `__iter__`
class MyIterableDataset(data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start
        self.start = start
        self.end = end

    def __iter__(self):
        worker_info = data.get_worker_info()
        if worker_info is None:  # single-process data loading, return the full iterator
            iter_start = self.start
            iter_end = self.end
        else:  # 
            # if it is in a worker process, split workload
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)
        return iter(range(iter_start, iter_end))

ds = MyIterableDataset(start=3, end=7)
print(list(data.DataLoader(ds, num_workers=0)))
print(list(data.DataLoader(ds, num_workers=1)))
print(list(data.DataLoader(ds, num_workers=20)))


# Example 2: splitting workload across all workers using :attr:`worker_init_fn`::
class MyIterableDataset(data.IterableDataset):
    def __init__(self, start, end):
        super(MyIterableDataset).__init__()
        assert end > start, "this example code only works with end > start"
        self.start = start
        self.end = end

    def __iter__(self):
        return iter(range(self.start, self.end))

# should give same set of data as range(3, 7), i.e., [3, 4, 5, 6].
ds = MyIterableDataset(start=3, end=7)

# Single-process loading
print(list(torch.utils.data.DataLoader(ds, num_workers=0)))

# Directly doing multi-process loading yields duplicate data
print(list(torch.utils.data.DataLoader(ds, num_workers=2)))

# Define a `worker_init_fn` that configures each dataset copy differently
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset  # the dataset copy in this worker process
    overall_start = dataset.start
    overall_end = dataset.end
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    dataset.start = overall_start + worker_id * per_worker
    dataset.end = min(dataset.start + per_worker, overall_end)
    ...

# Mult-process loading with the custom `worker_init_fn`
# Worker 0 fetched [3, 4].  Worker 1 fetched [5, 6].
print(list(torch.utils.data.DataLoader(ds, num_workers=2, worker_init_fn=worker_init_fn)))

# With even more workers
print(list(torch.utils.data.DataLoader(ds, num_workers=20, worker_init_fn=worker_init_fn)))


## data.TensorDataset()
`data.TensorDataset(*tensors: Tensor)`

包装了张量的数据集；通过沿着第一个维度的索引对张量进行检索，进而得到每个样本；参数`*tensors`应为若干第一个维度相同的张量；

**Type**:           type


**Example**
```python
inputs = torch.randint(0, 10, size=(6, 4))
targets = torch.arange(1, 7)
ds = data.TensorDataset(inputs, targets)
list(iter(ds))
"""
[(tensor([1, 2, 5, 5]), tensor(1)),
 (tensor([5, 2, 5, 9]), tensor(2)),
 (tensor([5, 9, 3, 7]), tensor(3)),
 (tensor([7, 6, 9, 5]), tensor(4)),
 (tensor([4, 3, 2, 8]), tensor(5)),
 (tensor([6, 8, 2, 4]), tensor(6))]
"""
```

## data.ConcatDataset()
`data.ConcatDataset(datasets: Iterable[Dataset])`

创建一个由多个数据集拼接而成的数据集，参数`datasets`应为数据集组成的列表；

**Type**:           type

## data.ChainDataset()
`data.ChainDataset(datasets: Iterable[Dataset])`

用于链接多个`IterableDataset`的数据集；与`ConcatDataset`不同，`ChainDataset`常用于对不同的现有**数据集流**进行组装；链接操作是动态完成的，因此用这个类连接大规模数据集会更加高效；`datasets`应为`IterableDataset`组成的可迭代对象

**Type**:           type     

## data.Subset()
`data.Subset(dataset: Dataset[T_co], indices: Sequence[int])`

由一个数据集的指定的索引构成的子集；其中参数`dataset`为原数据集，`indices`为指定的索引；

**Type**:           type

# 

# 

<center>

# Samplers
    
</center>


## utils.data.Sampler()

`utils.data.Sampler(data_source: Optional[Sized])`
所有`Sampler`的基类；所有继承此类的`Sampler`应定义`__iter__`方法，用以遍历所有数据集元素的索引；还应定义`__len__`方法用以返回`Sampler`子类的长度；需要注意的是，对于`DataLoader`类并没有严格要求对`__len__`方法进行实现，然而由于有众多涉及到`DataLoader`所含元素量的计算，进而建议自定义数据集时应对其进行实现；

**Type**:           type

**Subclasses**:     SequentialSampler, RandomSampler, SubsetRandomSampler, WeightedRandomSampler, BatchSampler, DistributedSampler, \_InfiniteConstantSampler



## utils.data.SubsetRandomSampler()
`utils.data.SubsetRandomSampler(indices: Sequence[int], generator=None)`

根据给定的索引组成的列表进行不放回 (without replacement) 随机采样；其中参数`indices`为索引组成的列表，`generator`为被采样的生成器


**Type**:           type


## utils.data.BatchSampler()
`utils.data.BatchSampler(sampler, batch_size, drop_last)`

对另一个采样器进行包装以生成一个 batch 的指数；这里参数`sampler`应为继承`Sampler`基类的对象或任何可迭代对象；`drop_last`则用于指明是否将最后样本数量不足指定``batch_size``的 batch 舍弃掉；


**Type**:           type

### Example

```python
seq_sampler = SequentialSampler(range(6, -6, -1))
list(BatchSampler(seq_sampler, batch_size=5, drop_last=False))
# => [[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11]]
```


## utils.data.RandomSampler()
```python
utils.data.RandomSampler(
    data_source: Sized,
    replacement: bool = False,
    num_samples: Optional[int] = None,
    generator=None
)
```
对样本进行随即采样；

**Args**
- data_source: 要进行采样的数据集
- replacement: True 时有放回的按需抽取样本，此时需指定`num_samples`；False 时则进行无放回采样；
- num_samples: 选取的样本的数量，默认`len(dataset)`；该参数只在`replacement`为 True 时起作用
- generator: 采样时使用的生成器


**Type**:           type


## utils.data.SequentialSampler()
`utils.data.SequentialSampler(data_source)`


Samples elements sequentially, always in the same order. `data_source`即为采样的数据集；


**Type**:           type

In [ ]:
utils.data.TensorDataset()